# Model Building - OCR


In [1]:
!pip install easyocr
!git clone https://github.com/JaidedAI/EasyOCR.git
!pip install -r EasyOCR/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 58.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 19.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Cloning into 'EasyOCR'...
remote: Enumerating objects: 2750, done.
remote: Counting objects: 100% (700/700), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 2750 (delta 646), reused 632 (delta 

In [2]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

Data Spliting

In [3]:
# Define dataset path
images_path = os.path.join('/kaggle/input/', "recognition-data")


# Read the annotation CSV file
df = pd.read_csv('/kaggle/input/recognition-annot/Licplatesrecognition_train.csv')
df.columns = ["img_id", "text"]  # Ensure correct column names

# Split data into Train (80%), Validation (10%), and Test (10%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Create directories for train, val, test sets
output_dirs = ["/kaggle/working/train", "/kaggle/working/val", "/kaggle/working/test"]
for d in output_dirs:
    os.makedirs(os.path.join(d, "images"), exist_ok=True)
    os.makedirs(os.path.join(d, "labels"), exist_ok=True)

# Function to move files to respective directories
def move_files(dataframe, split):
    for _, row in dataframe.iterrows():
        img_src = os.path.join(images_path, row["img_id"])
        img_dst = os.path.join(f"/kaggle/working/{split}/images", row["img_id"])
        shutil.copy(img_src, img_dst)
        
        # Save annotation
        label_dst = os.path.join(f"/kaggle/working/{split}/labels", row["img_id"].replace(".jpg", ".txt"))
        with open(label_dst, "w") as f:
            f.write(row["text"])

# Move images and labels to respective folders
move_files(train_df, "train")
move_files(val_df, "val")
move_files(test_df, "test")

print("Data split and saved successfully!")


Data split and saved successfully!


In [4]:
!pip install torch torchvision torchaudio
!pip install lmdb numpy
!pip install pillow



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 8.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import os


Defines a character to index mapping for digits (0-9) and an extra class for CTC loss blank.

In [6]:
# Character Mapping
characters = "0123456789"
char_to_index = {char: idx for idx, char in enumerate(characters)}
num_classes = len(characters) + 1 


##  CRNN Model
a CRNN model with CNN for feature extraction and bidirectional LSTM for sequence modeling.

In [7]:
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2) 
        self.rnn_input_size = 64 * 16 
        self.rnn = nn.LSTM(self.rnn_input_size, 128, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x))) 
        x = x.permute(0, 3, 2, 1).contiguous()  
        x = x.view(x.size(0), x.size(1), -1) 
        x, _ = self.rnn(x) 
        x = self.fc(x)
        return x


This dataset class that loads grayscale images and corresponding labels, encoding text into numerical sequences.

In [8]:
# Define Dataset Class
class OCRDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))

        self.labels = []
        for file in self.image_files:
            label_path = os.path.join(label_dir, file.replace(".jpg", ".txt"))
            if os.path.exists(label_path):
                label_text = open(label_path).read().strip()
                self.labels.append(label_text)
            else:
                self.labels.append("")

    def __len__(self):
        return len(self.image_files)

    def encode_label(self, label):
        if label == "":
            return torch.tensor([], dtype=torch.long) 
        return torch.tensor([char_to_index[char] for char in label if char in char_to_index], dtype=torch.long)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        label = self.labels[idx]
        image = Image.open(img_path).convert("L")  #grayscale
        if self.transform:
            image = self.transform(image)
        return image, self.encode_label(label)

In [9]:
# Load Dataset
transform = transforms.Compose([
    transforms.Resize((32, 128)),
    transforms.ToTensor()
])

dataset = OCRDataset("/kaggle/working/train/images", "/kaggle/working/train/labels", transform)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


Model Training

In [12]:
# Initialize Model & Training Components
model = CRNN(num_classes)
criterion = nn.CTCLoss(blank=num_classes-1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = torch.stack(images)  # Convert list to tensor
        
        # Filter out empty labels
        non_empty_labels = [l for l in labels if len(l) > 0]
        if not non_empty_labels:
            continue  # Skip batch if all labels are empty

        labels = torch.cat(non_empty_labels)  # Flatten labels
        
        outputs = model(images) 
        input_lengths = torch.full(size=(outputs.size(0),), fill_value=outputs.size(1), dtype=torch.long)
        target_lengths = torch.tensor([len(l) for l in non_empty_labels], dtype=torch.long)

        loss = criterion(outputs.permute(1, 0, 2), labels, input_lengths, target_lengths)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Save Model
torch.save(model.state_dict(), "/kaggle/working/custom_crnn.pth")

print("CRNN OCR Model Trained & Saved!")


Epoch 1/10, Loss: 3.0021700859069824
Epoch 2/10, Loss: 2.583437204360962
Epoch 3/10, Loss: 2.52703857421875
Epoch 4/10, Loss: 2.6779868602752686
Epoch 5/10, Loss: 2.430624008178711
Epoch 6/10, Loss: 2.5869107246398926
Epoch 7/10, Loss: 2.333925485610962
Epoch 8/10, Loss: 2.2667887210845947
Epoch 9/10, Loss: 2.481574535369873
Epoch 10/10, Loss: 2.4495689868927
CRNN OCR Model Trained & Saved!
